In [1]:
# !wget https://huseinhouse-data.s3-ap-southeast-1.amazonaws.com/Parl.+Replies_Verified-20191113T140600Z-001.zip
# !wget https://huseinhouse-data.s3-ap-southeast-1.amazonaws.com/Parl.+Replies_Verified-20191113T140600Z-002.zip
# !unzip Parl.+Replies_Verified-20191113T140600Z-001.zip
# !unzip Parl.+Replies_Verified-20191113T140600Z-002.zip

In [2]:
from glob import glob

pdfs = glob('Parl. Replies_Verified/**/*.pdf', recursive = True)
len(pdfs)

304

In [3]:
# !pip3 install tika --user

In [4]:
# wget https://www-eu.apache.org/dist/tika/tika-server-1.22.jar
# java -jar tika-server-1.22.jar --port 9997

In [5]:
from tika import parser
import tika
tika.TikaClientOnly = True

In [6]:
parsed = parser.from_file(pdfs[0], 'http://localhost:9997/')

In [7]:
import cleaning

from unidecode import unidecode

def clean(string):
    string = [cleaning.cleaning(s) for s in string]
    
    string = [s.strip() for s in string if 'tarikh' not in s.lower() and 'soalan no' not in s.lower()]
    string = [s for s in string if not ''.join(s.split()[:1]).isdigit() and '.soalan' not in s.lower() and 'jum ' not in s.lower()]
    string = [s for s in string if not s[:3].isdigit() and not s[-3:].isdigit()]
    return string

In [11]:
from tqdm import tqdm

outer = []

for k in tqdm(range(len(pdfs))):
    try:
        pdf = pdfs[k]
        parsed = parser.from_file(pdf, 'http://localhost:9997/')

        c = clean(parsed['content'].split('\n'))
        t, last = [], 0

        i = 0
        while i < len(c):
            text = c[i]

            if len(text) > 5:
                if len(text.split()) > 1:
                    t.append(text)
                last = i
            else:
                if len(t) and (i - last) > 2:
                    t.append('')
                    outer.extend(t)
                    t = []
                    last = i
                elif not len(t):
                    last = i

            i += 1

        if len(t):
            t.append('')
            outer.extend(t)
    except Exception as e:
        print(e)
        pass

  9%|▊         | 26/304 [00:22<03:02,  1.52it/s]

'NoneType' object has no attribute 'split'


 19%|█▉        | 59/304 [00:38<01:20,  3.06it/s]

'NoneType' object has no attribute 'split'


 21%|██        | 63/304 [00:40<01:56,  2.08it/s]

'NoneType' object has no attribute 'split'


 65%|██████▍   | 197/304 [01:53<00:59,  1.80it/s]

'NoneType' object has no attribute 'split'


100%|██████████| 304/304 [02:52<00:00,  1.77it/s]


In [12]:
len(outer)

960869

In [13]:
%%time

temp_vocab = list(set(cleaning.multiprocessing(outer, cleaning.unique_words)))

CPU times: user 351 ms, sys: 284 ms, total: 635 ms
Wall time: 744 ms


In [14]:
%%time

# important
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.duplicate_dots_marks_exclamations, list_mode = False)
print(len(temp_dict))

630
CPU times: user 74.9 ms, sys: 113 ms, total: 188 ms
Wall time: 555 ms


In [15]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 960869/960869 [00:02<00:00, 336637.78it/s]


In [16]:
%%time

# important
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_underscore, list_mode = False)
print(len(temp_dict))

200
CPU times: user 65.8 ms, sys: 116 ms, total: 181 ms
Wall time: 255 ms


In [18]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 960869/960869 [00:02<00:00, 340252.31it/s]


In [19]:
%%time

# important
temp_dict = cleaning.multiprocessing(outer, cleaning.isolate_spamchars, list_mode = False)
print(len(temp_dict))

0
CPU times: user 217 ms, sys: 184 ms, total: 402 ms
Wall time: 1.1 s


In [20]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_short_words, list_mode = False)
print(len(temp_dict))

4944
CPU times: user 66.3 ms, sys: 132 ms, total: 198 ms
Wall time: 271 ms


In [21]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 960869/960869 [00:02<00:00, 333845.07it/s]


In [22]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_long_words, list_mode = False)
print(len(temp_dict))

695
CPU times: user 99.3 ms, sys: 96.3 ms, total: 196 ms
Wall time: 238 ms


In [23]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 960869/960869 [00:02<00:00, 349343.64it/s]


In [24]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_ending_underscore, list_mode = False)
print(len(temp_dict))

230
CPU times: user 73.4 ms, sys: 120 ms, total: 193 ms
Wall time: 270 ms


In [25]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 960869/960869 [00:02<00:00, 339206.66it/s]


In [26]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_starting_underscore, list_mode = False)
print(len(temp_dict))

160
CPU times: user 66.4 ms, sys: 121 ms, total: 187 ms
Wall time: 264 ms


In [27]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 960869/960869 [00:02<00:00, 336362.07it/s]


In [28]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.end_punct, list_mode = False)
print(len(temp_dict))

70969
CPU times: user 294 ms, sys: 132 ms, total: 426 ms
Wall time: 533 ms


In [29]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 960869/960869 [00:03<00:00, 312211.61it/s]


In [30]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.start_punct, list_mode = False)
print(len(temp_dict))

30336
CPU times: user 167 ms, sys: 120 ms, total: 287 ms
Wall time: 344 ms


In [31]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 960869/960869 [00:03<00:00, 306990.19it/s]


In [32]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.join_dashes, list_mode = False)
print(len(temp_dict))

1238
CPU times: user 73.5 ms, sys: 128 ms, total: 201 ms
Wall time: 275 ms


In [33]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 960869/960869 [00:03<00:00, 315174.82it/s]


In [34]:
outer[-100:]

['Tuan Ahmad Baihaki Bin Atiqullah [ Kubang Kerian ] minta',
 "' PERDANA MENTERI menyatakan apakah status pembinaan",
 'Terminal Bersepadu Gombak ( ITT ) yang masih belum siap sehingga',
 '',
 "JAWAPAN : YB DATO ' SRI HAJAH NANCY SHUKRI",
 'MENTERI DI JABATAN PERDANA MENTERI',
 'Tuan Yang di-Pertua ,',
 '1 . Untuk makluman Ahli Yang Berhormat , pembinaan Terminal',
 'Bersepadu ( Integrated Transport Terminal - ITT ) Gombak telah',
 'dirancang oleh Kerajaan sebagai salah satu projek di bawah Bidang',
 'Keberhasilan Utama Negara ( National Key Result Area - NKRA ).',
 '2 . ITT Gombak ini akan menyediakan kemudahan bagi',
 'perkhidmatan bas ekspres untuk koridor Timur yang merangkumi',
 'negeri Pahang , Terengganu dan Kelantan . Lokasi ITT Gombak dipilih',
 'berdasarkan kebolehcapaiannya serta pengurangan kesesakan jalan',
 'raya kepada orang ramai dan kesepaduannya (intergration ) dengan',
 'stesen LRT Gombak apabila ia siap dibina kelak .',
 '3 . Pada masa ini , kerja-kerja awalan sedan

In [35]:
with open('dumping-parliament.txt', 'w') as fopen:
    fopen.write('\n'.join(outer))